**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-4z7jhn_5
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-4z7jhn_5
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-4z7jhn_5 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-4z7jhn_5 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,9f895c32eb7ef473c8c03da76e99a803dfcab41a,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-07-26 21:00:00+00:00,2013-07-26 21:00:00+00:00,60,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
1,f4c42db73e32ec8fd4814e5e22b4a1947d08c080,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-07-26 21:00:00+00:00,2013-07-26 21:00:00+00:00,0,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
2,3d147cb3f85483a1c00c7d7e27742712bf1e8f8c,38205a7b4cb9cdabd0d6ee8c4c6b115c53e7bd7b06645a...,2014-05-09 14:45:00+00:00,2014-05-09 14:45:00+00:00,120,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,None,NaN,NaN,None,NaN,NaN,None
3,522ac1df4121f3249d82761e1b37c91521f4e923,ac1045ae61a5226a9216bd9641b458bd9d7e7da0680ada...,2013-12-06 22:15:00+00:00,2013-12-06 22:30:00+00:00,840,0.35,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
4,93eb46600c3222a92d7d78cbac6ac35c42be9a77,78d45b51ff03d70f0ba2afda334acf93f678a179aaaaca...,2014-03-06 15:30:00+00:00,2014-03-06 15:45:00+00:00,1500,1.39,NaN,1.703198e+10,NaN,76.0,...,NaN,NaN,Credit Card,None,NaN,NaN,None,41.979071,-87.90304,POINT (-87.9030396611 41.9790708201)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [6]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING 
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-27,81378.428571
1,2016-01-13,85459.285714
2,2016-01-29,79909.857143
3,2016-02-29,92275.285714
4,2016-01-14,86916.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [9]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER(
                            PARTITION BY pickup_community_area
                            ORDER BY trip_start_timestamp
                        ) as trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,30.0,2013-10-03 01:45:00+00:00,2013-10-03 02:00:00+00:00,1
1,30.0,2013-10-03 02:15:00+00:00,2013-10-03 02:30:00+00:00,2
2,30.0,2013-10-03 09:30:00+00:00,2013-10-03 09:45:00+00:00,3
3,30.0,2013-10-03 11:00:00+00:00,2013-10-03 11:45:00+00:00,4
4,30.0,2013-10-03 12:00:00+00:00,2013-10-03 12:15:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [12]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp,1) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,1c488d9782accfc14ecb6a822a1478d87eb26e6b53588b...,2013-10-03 13:45:00+00:00,2013-10-03 13:30:00+00:00,375.0
1,1dee29743ced81af2e20060425fab8b8abe686f9659e72...,2013-10-03 23:00:00+00:00,2013-10-03 23:30:00+00:00,165.0
2,2ec5c47128a98f946f5f1382680e6e48752eb935542ad1...,2013-10-03 18:15:00+00:00,2013-10-03 18:15:00+00:00,450.0
3,2ec5c47128a98f946f5f1382680e6e48752eb935542ad1...,2013-10-03 21:45:00+00:00,2013-10-03 21:45:00+00:00,210.0
4,2fc72622ea88354ea99d9bd64750fed38df2bb87a79f88...,2013-10-03 22:30:00+00:00,2013-10-03 22:30:00+00:00,-30.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*